<a href="https://colab.research.google.com/github/DougChul/RNA/blob/Colab/Most_Basic_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import

In [ ]:
import os
import sys
import pickle
import random
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import math


In [ ]:
#set seed for everything
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
print(torch.__version__)
print(pd.__version__)

2.6.0+cu124
2.2.2


## Config

In [ ]:
config = {
    "seed": 0,
    "cutoff_date": "2020-01-01",
    "test_cutoff_date": "2022-05-01",
    "max_len": 384,  ##
    "batch_size": 1,
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "mixed_precision": "bf16",
    "model_config_path": "../working/configs/pairwise.yaml",  # Adjust path as needed
    "epochs": 50,  ##
    "loss_power_scale": 1.0,
    "max_cycles": 1,
    "grad_clip": 0.1,
    "gradient_accumulation_steps": 1,
    "d_clamp": 30,
    "max_len_filter": 384,
    "min_len_filter": 10,
    "structural_violation_epoch": 50,
    "balance_weight": False,
    "n_tokens": 4,
    "d_model": 256,  ##
    "n_heads": 8,
    "dropout": 0.1,
    "d_ff": 1024, ##
    "norm_ratio": 0.3, ##
    "n_layers": 30 ##
}

## Set Sample Data To make Model

In [ ]:
os.getcwd()

'/content'

In [ ]:

# folder_path = '/content/drive/MyDrive/RNA/stanford-rna-3d-folding'

# ## Select Data
# # set_data_fomat = '5models'
# set_data_fomat = 'v2'
# # set_data_fomat = 'v1'

# if set_data_fomat == '5models':
#     train_labels = pd.read_csv(os.path.join(folder_path, 'pdb_labels_5models.csv'))
#     train_sequences = pd.read_csv(os.path.join(folder_path, 'pdb_sequences_5models.csv'))
# elif set_data_fomat == 'v2':
#     train_labels = pd.read_csv(os.path.join(folder_path, 'train_labels.v2.csv'))
#     train_sequences = pd.read_csv(os.path.join(folder_path, 'train_sequences.v2.csv'))
# elif set_data_fomat == 'v1':
#     train_labels = pd.read_csv(os.path.join(folder_path, 'train_labels.csv'))
#     train_sequences = pd.read_csv(os.path.join(folder_path, 'train_sequences.csv'))
# else:
#     raise ValueError("Invalid set_data_fomat")


# print(train_labels.head())

# print(train_sequences.head())


In [ ]:
# train_labels["pdb_id"] = train_labels["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
# # train_sequences["pdb_id"]

In [ ]:
# all_xyz=[]

# test_sample = False
# count = 0

# for pdb_id in tqdm(train_sequences['target_id']):
#     df = train_labels[train_labels["pdb_id"]==pdb_id]
#     #break
#     # xyz=df[['x_1','y_1','z_1','x_2','y_2','z_2','x_3','y_3','z_3','x_4','y_4','z_4','x_5','y_5','z_5',]].to_numpy().astype('float32')
#     xyz=df[['x_1','y_1','z_1']].to_numpy().astype('float32')
#     xyz[xyz<-1e17]=float('Nan');
#     all_xyz.append(xyz)
#     if test_sample == True:
#       count += 1
#       if count == 100:
#         break
# # all_xyz[13]

### filter data

In [ ]:
# # filter the data
# # Filter and process data
# filter_nan = []
# max_len = 0
# filter_ratio = 0 # All data are valid
# for xyz in all_xyz:
#     if len(xyz) > max_len:
#         max_len = len(xyz)

#     filter_nan.append((np.isnan(xyz).mean() <= filter_ratio) & \
#                       (len(xyz)<config['max_len_filter']) & \
#                       (len(xyz)>config['min_len_filter']))


# print(f"Longest sequence in train: {max_len}")

# filter_nan = np.array(filter_nan)
# non_nan_indices = np.arange(len(filter_nan))[filter_nan]
# print('remain sequences:', len(non_nan_indices))
# train_sequences = train_sequences.loc[non_nan_indices].reset_index(drop=True)
# non_nan_xyz=[all_xyz[i] for i in non_nan_indices]

In [ ]:
# #pack data into a dictionary

# data={
#       "pdb_id":train_sequences['target_id'].to_list(),
#       "sequence":train_sequences['sequence'].to_list(),
#       "temporal_cutoff": train_sequences['temporal_cutoff'].to_list(),
#       "description": train_sequences['description'].to_list(),
#       "all_sequences": train_sequences['all_sequences'].to_list(),
#       "xyz": non_nan_xyz
# }
# print(data['pdb_id'][2])
# print(data['sequence'][1])
# print(data['temporal_cutoff'][1])
# # print(data['description'][1])
# # print(data['all_sequences'][1])
# # print(data['xyz'][1])

In [ ]:
# with open('/content/drive/MyDrive/RNA/stanford-rna-3d-folding/processed_data.v2_max=384.pkl', 'wb') as f:
#     pickle.dump(data, f)
# print("데이터가 processed_data.pkl 파일로 저장되었습니다.")

In [ ]:
## Load Preprocess DAta

with open('/content/drive/MyDrive/RNA/stanford-rna-3d-folding/processed_data.v2_max=384.pkl', 'rb') as f:
    data = pickle.load(f)

### Split Train / Validation set

In [ ]:
# Split data into train and test
all_index = np.arange(len(data['sequence']))
cutoff_date = pd.Timestamp(config['cutoff_date'])
test_cutoff_date = pd.Timestamp(config['test_cutoff_date'])
train_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) <= cutoff_date]
test_index = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) > cutoff_date and pd.Timestamp(d) <= test_cutoff_date]

### To Pytorch Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from ast import literal_eval

def get_ct(bp,s):
    ct_matrix=np.zeros((len(s),len(s)))
    for b in bp:
        ct_matrix[b[0]-1,b[1]-1]=1
    return ct_matrix

class RNA3D_Dataset(Dataset):
    def __init__(self,indices,data):
        self.indices=indices
        self.data=data
        self.tokens={nt:i for i,nt in enumerate('ACGU')}
        # self.tokens = {'A': 2, 'U':-2, 'G':3, 'C':-3}

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):

        idx=self.indices[idx]
        sequence=[self.tokens[nt] for nt in (self.data['sequence'][idx])]
        sequence=np.array(sequence)
        sequence=torch.tensor(sequence)

        #get C1' xyz
        xyz=self.data['xyz'][idx]
        xyz=torch.tensor(np.array(xyz))


        if len(sequence)>config['max_len']:
            crop_start=np.random.randint(len(sequence)-config['max_len'])
            crop_end=crop_start+config['max_len']

            sequence=sequence[crop_start:crop_end]
            xyz=xyz[crop_start:crop_end]
        #center at first atom if first atom does not exit go until it does
        for i in range(len(xyz)):
            if (~torch.isnan(xyz[i])).all():
                break
        xyz=xyz-xyz[i]

        return {'pbd_id':self.data['pdb_id'][idx],
            'sequence':sequence,
                'xyz':xyz}

In [ ]:

train_dataset=RNA3D_Dataset(train_index,data)
val_dataset=RNA3D_Dataset(test_index,data)


In [ ]:
train_loader=DataLoader(train_dataset,batch_size=1,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=1,shuffle=False)

# print(train_dataset[2]['xyz'][6])


## Model definition

### Example with one data

In [ ]:
test_sample = True

if test_sample == True:
  from torch.utils.data import Subset

  target_index = 0
  train_dataset = Subset(train_loader.dataset, [target_index])
  train_loader = DataLoader(train_dataset, batch_size=1)

  val_dataset = Subset(val_loader.dataset, [target_index])
  val_loader = DataLoader(val_dataset, batch_size=1)

print(len(train_dataset[0]['sequence']))
print(train_dataset[0])
print(len(train_loader))
print(len(val_loader))
# print(f"새로운 데이터셋 크기: {len(single_data_loader.dataset)}")
# print(f"새로운 DataLoader의 배치 개수: {len(single_data_loader)}")

12
{'pbd_id': '3J0Q_3', 'sequence': tensor([2, 1, 1, 0, 2, 3, 2, 0, 0, 0, 3, 0]), 'xyz': tensor([[  0.0000,   0.0000,   0.0000],
        [ -1.0760,  -3.3350,  -3.4670],
        [ -4.4130,  -6.6660,  -7.6410],
        [ -7.0260,  -0.6310,  -9.7380],
        [ -3.1140,   3.6950, -11.0880],
        [ -2.5960,   8.3430, -14.4300],
        [  1.2760,  10.8900, -15.9760],
        [  7.1280,  12.5880, -19.2280],
        [  8.5000,   9.1200, -14.2150],
        [  4.5950,   6.0570,  -8.3870],
        [  6.2800,   5.2940,  -2.2990],
        [  6.1480,   2.7090,   3.4880]])}
1
1


### Check Pairwise

In [ ]:
# import RNA
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# src = torch.tensor([2, 2, 1, 2, 3, 0, 0, 2, 2, 0, 3, 3, 0, 1, 1, 3, 0, 3, 2, 1, 1], dtype=torch.long)
# # AUGC를 0123으로 매핑
# nucleotide_map = {0: 'A', 1: 'C', 2: 'G', 3: 'U'}

# str_A = ''.join([nucleotide_map[x.item()] for x in src])

# print(len(str_A))
# print(str_A)

# md = RNA.md()

# fc = RNA.fold_compound(str_A, md)

# # predict Minmum Free Energy and corresponding secondary structure
# # (ss, mfe) =
# # print(fc.mfe())
# fc.pf()
# B = torch.tensor(fc.bpp())

# # B = torch.tensor(B)
# B = B[1:,1:]
# print(type(B))
# print(B.shape)
# B = B + B.T
# # B = np.array(B, dtype=np.bool())
# # print(B)

# plt.figure(figsize=(5,4))
# sns.heatmap(B, cmap='viridis', annot=False)  # annot=True로 설정하면 각 셀에 값 표시
# plt.title('Pairwise Contact Probability Matrix')
# plt.show()

# # def get_pairwise_features(src, seq_len, d_model):
# #     nucleotide_map = {0: 'A', 1: 'C', 2: 'G', 3: 'U'}

# #     # print("pairwise",src)
# #     src = src.squeeze(0)
# #     str_seq = ''.join([nucleotide_map[x.item()] for x in src])
# #     # print("str_seq",str_seq)

# #     md = RNA.md()
# #     fc = RNA.fold_compound(str_seq, md)
# #     fc.pf() ##???

# #     pair_matrix = torch.tensor(fc.bpp(), dtype=torch.float32)
# #     # print("pair_matrix",pair_matrix)
# #     pair_matrix = pair_matrix[1:,1:] # remove first row and column 0 index in bpp is always 0
# #     pair_matrix = pair_matrix + pair_matrix.T # symmetric matrix

# #     pair_matrix = pair_matrix.unsqueeze(0).unsqueeze(0)
# #     pair_matrix = F.interpolate(pair_matrix, size=(d_model, d_model), mode='bilinear', align_corners=False)
# #     # pair_matrix = F.interpolate(pair_matrix, size=(d_model, d_model), mode='nearest', align_corners=False)

# #     pair_matrix = pair_matrix.squeeze()
# #     # print("pair_matrix",pair_matrix.shape)

# #     return pair_matrix

### MSA Module

In [ ]:
class OuterProductMean(nn.Module):
    def __init__(self, in_dim=256, dim_msa=32, pairwise_dim=128):
        super(OuterProductMean, self).__init__()
        self.proj_down1 = nn.Linear(in_dim, dim_msa)
        self.proj_down2 = nn.Linear(dim_msa ** 2, pairwise_dim)

    def forward(self,seq_rep, pair_rep=None):
        seq_rep=self.proj_down1(seq_rep)
        outer_product = torch.einsum('bid,bjc -> bijcd', seq_rep, seq_rep)
        outer_product = rearrange(outer_product, 'b i j c d -> b i j (c d)')
        outer_product = self.proj_down2(outer_product)

        if pair_rep is not None:
            outer_product=outer_product+pair_rep

        return outer_product

### Back Bone

In [ ]:
# import RNA
import torch.nn.functional as F
from einops import rearrange, repeat, reduce

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=config['max_len']):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # print('x', x.shape)
        # print('pe', self.pe[:, : x.size(1)])
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)


class relpos(nn.Module):

    def __init__(self, dim=64):
        super(relpos, self).__init__()
        self.linear = nn.Linear(33, dim)

    def forward(self, src):
        L=src.shape[1]
        res_id = torch.arange(L).to(src.device).unsqueeze(0)
        device = res_id.device
        bin_values = torch.arange(-16, 17, device=device)
        #print((bin_values))
        d = res_id[:, :, None] - res_id[:, None, :]
        bdy = torch.tensor(16, device=device)
        d = torch.minimum(torch.maximum(-bdy, d), bdy)
        d_onehot = (d[..., None] == bin_values).float()
        #print(d_onehot.sum(dim=-1).min())
        assert d_onehot.sum(dim=-1).min() == 1
        p = self.linear(d_onehot)
        return p

# class ConstrainedPositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_relative_position=32, constrained_position=4):
#         super().__init__()
#         self.max_relative_position = max_relative_position
#         self.relative_embedding = nn.Embedding(2 * max_relative_position + 1, d_model)
#         self.constrained_position = constrained_position

#     def forward(self, seq_len):

#         return pos_encoding(seq_len, self.d_model)


class EmbedSequence(nn.Module):
    def __init__(self, d_model, n_tokens = config['n_tokens']):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(n_tokens, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout=0.1)
        # self.pos_encoding = PosEncoding(seq_len, d_model)

    def forward(self, src):
        src = self.embedding(src.long())
        return src + self.pos_encoder(src)


class MultiheadAtt(nn.Module):
    def __init__(self, d_model, n_heads=8, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.dropout = nn.Dropout(dropout)

        self.w_q = nn.Linear(d_model, n_heads * self.d_head)
        self.w_k = nn.Linear(d_model, n_heads * self.d_head)
        self.w_v = nn.Linear(d_model, n_heads * self.d_head)

        self.w_o = nn.Linear(n_heads * self.d_head, d_model)

    def forward(self, query, key, value):
        # print('query.shape',query.shape)
        batch_size, length, d_model = query.size()

        q = self.w_q(query)
        k = self.w_k(key)
        v = self.w_v(value)

        q = q.view(batch_size, length, self.n_heads, self.d_head)
        k = k.view(batch_size, length, self.n_heads, self.d_head)
        v = v.view(batch_size, length, self.n_heads, self.d_head)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_head)
        scores = self.dropout(F.softmax(scores, dim=-1))

        output = torch.matmul(scores, v)
        output = output.transpose(1, 2).contiguous().view(batch_size, length, -1)
        output = self.w_o(output)


        return output


class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()

        self.dropout = nn.Dropout(dropout)

        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.w_1(x)
        x = self.activation(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.w_2(x)

        return x


class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0, norm_type='post_ln'):
        super().__init__()

        self.d_ff = config['d_ff']

        if (norm_type == 'post_ln') or (norm_type == 'pre_ln'):
            self.norm_type = norm_type
        else:
            raise ValueError("Invalid norm_type")

        self.norm_0 = nn.LayerNorm(d_model)
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        self.attention = MultiheadAtt(d_model, n_heads, dropout)
        self.feedforward = FeedForward(d_model, self.d_ff, dropout)
        self.activation = nn.ReLU()

    def forward(self, src, pairwise_feature=None):
        # src = residual_connection(src, self.encoder(src))
        norm_type = self.norm_type


        if norm_type == 'post_ln':

            src = self.norm_0(src)
            src = src + self.attention(src,src,src)  #residual conn
            src = self.norm_1(src)
            src = src + self.feedforward(src)  #residual conn
            src = self.norm_2(src)


        elif norm_type == 'pre_ln':
            src_temp = src
            src = self.norm_0(src)
            src = self.attention(src,src,src) + src_temp
            # src = self.activation(src)
            src_temp = src
            src = self.norm_1(src)
            src = self.feedforward(src) + src_temp
            # src = self.activation(src)
            src = self.norm_2(src)

        return src


class WuSubSol(nn.Module):
    def __init__(self, config):
        super(WuSubSol, self).__init__()
        self.config = config
        self.n_layers = config['n_layers']
        self.d_model = config['d_model']

        self.norm_ratio = config['norm_ratio']
        if (self.n_layers * self.norm_ratio).is_integer():
            self.n_post_ln = int(self.n_layers * self.norm_ratio)
        else:
            print("Invalid norm_ratio")
            exit()

        self.embedding = EmbedSequence(self.d_model)


        self.encoder_layers = []
        count_post_ln, count_pre_ln = 0,0
        for i in range(self.n_post_ln):
            self.encoder_layers.append(EncoderLayer(self.d_model, config['n_heads'], config['dropout'], 'post_ln'))
            count_post_ln += 1
        for i in range(self.n_layers - self.n_post_ln):
            self.encoder_layers.append(EncoderLayer(self.d_model, config['n_heads'], config['dropout'], 'pre_ln'))
            count_pre_ln += 1

        print(f"{count_post_ln} post_ln layers and {count_pre_ln} pre_ln layers")

        self.encoder_layers = nn.ModuleList(self.encoder_layers)

        self.final_linear = nn.Linear(self.d_model, 3)

        print(f"{self.n_layers} layers of encoder constructed")

    def forward(self, src):
        pairwise_feature = None
        src = self.embedding(src) # L*d_model
        # print('after embedding', src.shape)


        for i,layer in enumerate(self.encoder_layers):
            # print("before",src.shape)
            src = layer(src, pairwise_feature)
            # print("after",src.shape)

        src = self.final_linear(src).squeeze()
        for i in range(len(src)):
            if (~torch.isnan(src[i])).all():
                break
        src=src-src[i]

        # print('final tensor-Shape : ', src.shape)

        return src



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = WuSubSol(config)
model.to(device)
print(model)


9 post_ln layers and 21 pre_ln layers
30 layers of encoder constructed
WuSubSol(
  (embedding): EmbedSequence(
    (embedding): Embedding(4, 256)
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (encoder_layers): ModuleList(
    (0-29): 30 x EncoderLayer(
      (norm_0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (attention): MultiheadAtt(
        (dropout): Dropout(p=0.1, inplace=False)
        (w_q): Linear(in_features=256, out_features=256, bias=True)
        (w_k): Linear(in_features=256, out_features=256, bias=True)
        (w_v): Linear(in_features=256, out_features=256, bias=True)
        (w_o): Linear(in_features=256, out_features=256, bias=True)
      )
      (feedforward): FeedForward(
        (dropout): Dropout(p=0.1, inplace=False)
        (w_1): Linear(in_features=256, o

## Training Loop

### Define Loss Function

In [ ]:
def calculate_distance_matrix(X,Y,epsilon=1e-4):
    return (torch.square(X[:,None]-Y[None,:])+epsilon).sum(-1).sqrt()

def dRMAE(pred_x,
          pred_y,
          gt_x,
          gt_y,
          epsilon=1e-4,Z=10,d_clamp=None):
    pred_dm=calculate_distance_matrix(pred_x,pred_y)
    gt_dm=calculate_distance_matrix(gt_x,gt_y)

    mask=~torch.isnan(gt_dm)
    mask[torch.eye(mask.shape[0]).bool()]=False

    rmsd=torch.abs(pred_dm[mask]-gt_dm[mask])

    return rmsd.mean()/Z

def align_svd_mae(input, target, Z=10):
    """
    Aligns the input (Nx3) to target (Nx3) using SVD-based Procrustes alignment
    and computes RMSD loss.

    Args:
        input (torch.Tensor): Nx3 tensor representing the input points.
        target (torch.Tensor): Nx3 tensor representing the target points.

    Returns:
        aligned_input (torch.Tensor): Nx3 aligned input.
        rmsd_loss (torch.Tensor): RMSD loss.
    """
    # print('input-Shape', input.shape)
    # print('output-Shape', target.shape)
    # target = target[:, :3]
    assert input.shape == target.shape, "Input and target must have the same shape"

    #mask
    mask=~torch.isnan(target.sum(-1))

    input=input[mask]
    target=target[mask]

    # Compute centroids
    centroid_input = input.mean(dim=0, keepdim=True)
    centroid_target = target.mean(dim=0, keepdim=True)

    # Center the points
    input_centered = input - centroid_input.detach()
    target_centered = target - centroid_target

    # Compute covariance matrix
    cov_matrix = input_centered.T @ target_centered

    # SVD to find optimal rotation
    U, S, Vt = torch.svd(cov_matrix)

    # Compute rotation matrix
    R = Vt @ U.T

    # Ensure a proper rotation (det(R) = 1, no reflection)
    if torch.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt @ U.T

    # Rotate input
    aligned_input = (input_centered @ R.T.detach()) + centroid_target.detach()

    # # Compute RMSD loss
    # rmsd_loss = torch.sqrt(((aligned_input - target) ** 2).mean())

    # rmsd_loss = torch.sqrt(((aligned_input - target) ** 2).mean())

    # return aligned_input, rmsd_loss
    return torch.abs(aligned_input-target).mean()/Z

def align_svd_rmsd(input, target):
    """
    Aligns the input (Nx3) to target (Nx3) using SVD-based Procrustes alignment
    and computes RMSD loss.

    Args:
        input (torch.Tensor): Nx3 tensor representing the input points.
        target (torch.Tensor): Nx3 tensor representing the target points.

    Returns:
        aligned_input (torch.Tensor): Nx3 aligned input.
        rmsd_loss (torch.Tensor): RMSD loss.
    """
    # print('input-Shape', input.shape)
    # print('output-Shape', target.shape)
    # target = target[:,:3]
    assert input.shape == target.shape, "Input and target must have the same shape"

    #mask
    mask=~torch.isnan(target.sum(-1))


    input=input[mask]
    target=target[mask]

    # Compute centroids
    centroid_input = input.mean(dim=0, keepdim=True)
    centroid_target = target.mean(dim=0, keepdim=True)

    # Center the points
    input_centered = input - centroid_input.detach()
    target_centered = target - centroid_target

    # Compute covariance matrix
    cov_matrix = input_centered.T @ target_centered

    # SVD to find optimal rotation
    U, S, Vt = torch.svd(cov_matrix)

    # Compute rotation matrix
    R = Vt @ U.T

    # Ensure a proper rotation (det(R) = 1, no reflection)
    if torch.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt @ U.T

    # Rotate input
    aligned_input = (input_centered @ R.T.detach()) + centroid_target.detach()

    # # Compute RMSD loss
    # rmsd_loss = torch.sqrt(((aligned_input - target) ** 2).mean())

    # rmsd_loss = torch.sqrt(((aligned_input - target) ** 2).mean())

    # return aligned_input, rmsd_loss
    return torch.square(aligned_input-target).mean().sqrt()

def compute_lddt(ground_truth_atoms, predicted_atoms, cutoff=30.0, thresholds=[1.0, 2.0, 4.0, 8.0]):
    """
    Computes the lDDT score between ground truth and predicted atoms.

    Parameters:
        ground_truth_atoms (np.array): Nx3 array of ground truth atom coordinates.
        predicted_atoms (np.array): Nx3 array of predicted atom coordinates.
        cutoff (float): Distance cutoff in Ångstroms to consider neighbors. Default is 30 Å.
        thresholds (list): List of thresholds in Ångstroms for the lDDT computation. Default is [0.5, 1.0, 2.0, 4.0].

    Returns:
        float: The lDDT score.
    """
    # Number of atoms
    num_atoms = ground_truth_atoms.shape[0]

    # Initialize array to store lDDT fractions for each threshold
    fractions = np.zeros(len(thresholds))

    for i in range(num_atoms):
        # Get the distances from atom i to all other atoms for both ground truth and predicted atoms
        gt_distances = np.linalg.norm(ground_truth_atoms[i] - ground_truth_atoms, axis=1)
        pred_distances = np.linalg.norm(predicted_atoms[i] - predicted_atoms, axis=1)

        # print(gt_distances)
        # print(pred_distances)
        # exit()
        # Apply the cutoff to consider only distances within the cutoff range
        mask = (gt_distances > 0) & (gt_distances < cutoff)

        # Calculate the absolute difference between ground truth and predicted distances
        distance_diff = np.abs(gt_distances[mask] - pred_distances[mask])

        # Filter out any NaN values from the distance difference calculation
        valid_mask = ~np.isnan(distance_diff)
        distance_diff = distance_diff[valid_mask]

        # Compute the fractions for each threshold
        for j, threshold in enumerate(thresholds):
            if len(distance_diff)>0:
                fractions[j] += np.mean(distance_diff < threshold)
    # print(fractions)
    # print(num_atoms)

    # Average the fractions over the number of atoms
    fractions /= num_atoms

    # The final lDDT score is the average of these fractions
    lddt_score = np.mean(fractions)

    return lddt_score

### Training SetUp

In [ ]:
from tqdm import tqdm

epochs=config['epochs']
# cos_epoch=config['cos_epoch']
cos_epoch=0


best_loss=np.inf
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0, lr=config['learning_rate']) #no weight decay following AF

batch_size=1

#for cycle in range(2):

criterion=torch.nn.CrossEntropyLoss(reduction='none')

#scaler = GradScaler()

schedule=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(epochs-cos_epoch)*len(train_loader)//batch_size)


### Train

In [ ]:
version_name = 'test_set_1'

best_val_loss=99999999999
loss_df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss'])

# print(type(train_loader))

# epochs = 20

for epoch in range(epochs):
    model.train()
    tbar=tqdm(train_loader)
    total_loss=0
    train_loss=0

    for idx, batch in enumerate(tbar):
        #try:
        sequence=batch['sequence'].cuda()
        gt_xyz=batch['xyz'].squeeze().cuda()

        mask=~torch.isnan(gt_xyz)

        gt_xyz[torch.isnan(gt_xyz)]=0

        pred_xyz = model(sequence).squeeze()
        if epoch == epochs-1:
              print(pred_xyz)
              print(gt_xyz)

        loss = dRMAE(pred_xyz,pred_xyz,gt_xyz,gt_xyz) + align_svd_mae(pred_xyz, gt_xyz)

        if loss!=loss:
            stop

        try:
          (loss/batch_size).backward()
        except:
          print(gt_xyz.shape)
        # (loss/batch_size).backward()

        if (idx+1)%batch_size==0 or idx+1 == len(tbar):

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
            optimizer.zero_grad()
            # scaler.scale(loss/batch_size).backward()
            # scaler.unscale_(optimizer)
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            # scaler.step(optimizer)
            # scaler.update()

            if (epoch+1)>cos_epoch:
                schedule.step()

        total_loss += loss
        train_loss = total_loss/(idx+1)

        tbar.set_description(f"Epoch {epoch + 1} Loss: {total_loss/(idx+1)}")

    ### Validation
    tbar=tqdm(val_loader)

    model.eval()
    val_preds=[]
    val_loss=0

    for idx, batch in enumerate(tbar):
        sequence=batch['sequence'].cuda()
        gt_xyz=batch['xyz'].squeeze().cuda()

        with torch.no_grad():
            pred_xyz=model(sequence).squeeze()
            if epoch == epochs-1:
              print(pred_xyz)
              print(gt_xyz)

            loss = dRMAE(pred_xyz,pred_xyz,gt_xyz,gt_xyz) + align_svd_mae(pred_xyz, gt_xyz)

        val_loss+=loss
        val_preds.append([gt_xyz.cpu().numpy(),pred_xyz.cpu().numpy()])

    val_loss=val_loss/len(tbar)
    print(f"val loss: {val_loss}")

    save_df = pd.DataFrame({'epoch': [epoch], 'train_loss': [train_loss.cpu().item()], 'val_loss': [val_loss.cpu().item()]})
    loss_df = pd.concat([loss_df, save_df], ignore_index=True)

    ## Check Best Loss .pt and Save
    if val_loss<best_val_loss:
        best_val_loss=val_loss
        best_preds=val_preds
        torch.save(model.state_dict(),f'/content/drive/MyDrive/RNA/WuSubSol/Save_Data/{version_name}_{epochs}_best.pt')

    # 1.053595052265986 train loss after epoch 0
torch.save(model.state_dict(),f'/content/drive/MyDrive/RNA/WuSubSol/Save_Data/{version_name}_{epochs}_last.pt')

# Save Loss,

loss_df.to_csv(f'/content/drive/MyDrive/RNA/WuSubSol/Save_Data/{version_name}_{epochs}_loss.csv', index=False)

print('save complete')

100%|██████████| 1/1 [00:00<00:00, 36.99it/s]
<ipython-input-25-ca257142f279>:86: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  loss_df = pd.concat([loss_df, save_df], ignore_index=True)


val loss: 3.4327447414398193


100%|██████████| 1/1 [00:00<00:00, 40.91it/s]


val loss: 3.4241013526916504


100%|██████████| 1/1 [00:00<00:00, 40.41it/s]


...

val loss: 3.167633056640625


Epoch 50 Loss: 1.2419521808624268: 100%|██████████| 1/1 [00:00<00:00,  8.19it/s]


tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.2161,  0.9230,  5.4712],
        [-0.0511,  1.3395,  5.7365],
        [-9.1964, -2.0464,  2.1981],
        [ 0.5511,  0.3403,  0.6324],
        [ 0.2743,  2.1257,  1.9717],
        [-0.2609, -0.5264, -0.0494],
        [-9.5399, -2.5150,  2.6933],
        [-9.4151, -2.2497,  2.4974],
        [-9.5581, -2.2083,  2.0122],
        [ 0.6165,  1.8323,  2.1593],
        [-9.4999, -2.3287,  2.1330]], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
tensor([[  0.0000,   0.0000,   0.0000],
        [ -1.0760,  -3.3350,  -3.4670],
        [ -4.4130,  -6.6660,  -7.6410],
        [ -7.0260,  -0.6310,  -9.7380],
        [ -3.1140,   3.6950, -11.0880],
        [ -2.5960,   8.3430, -14.4300],
        [  1.2760,  10.8900, -15.9760],
        [  7.1280,  12.5880, -19.2280],
        [  8.5000,   9.1200, -14.2150],
        [  4.5950,   6.0570,  -8.3870],
        [  6.2800,   5.2940,  -2.2990],
        [  6.1480,   2.7090,   3.4880]], device='cuda:0')


100%|██████████| 1/1 [00:00<00:00, 29.64it/s]


tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.5535e-02, -1.2110e-02, -1.1210e-03],
        [-6.5715e+00, -1.3966e+00,  1.3455e+00],
        [-7.5796e-02,  9.1916e-01,  3.4378e+00],
        [-9.3508e-02,  9.0464e-01,  3.4325e+00],
        [ 6.9673e-01,  1.3625e+00,  1.3570e+00],
        [-6.6602e+00, -1.3338e+00,  1.4451e+00],
        [-1.7041e-01,  9.1772e-01,  3.3394e+00],
        [ 6.4403e-01,  1.2785e+00,  1.4280e+00],
        [-6.5306e+00, -1.2974e+00,  1.4343e+00],
        [-1.3569e-01,  9.6197e-01,  3.3590e+00],
        [ 4.1699e-02,  2.5668e-02,  2.4292e-02],
        [-6.7032e+00, -1.2785e+00,  1.4380e+00],
        [ 2.0491e-02,  6.6305e-02,  9.3706e-03],
        [-3.0857e-01,  1.0066e+00,  3.3743e+00],
        [ 3.3944e-03,  3.1775e-02,  2.3566e-02],
        [-2.9409e-01,  9.6661e-01,  3.4242e+00],
        [-2.6359e-01,  9.3664e-01,  3.4560e+00],
        [-6.4876e+00, -1.3059e+00,  1.3606e+00],
        [ 5.0856e-01,  1.3257e+00,  1.4153e+00],
        [ 4.6202e-01

# Plot Loss Graph

In [ ]:
import pandas as pd
import plotly.graph_objects as go
epochs=config['epochs']
# CSV 로드
df = pd.read_csv(f'/content/drive/MyDrive/RNA/WuSubSol/Save_Data/{version_name}_{epochs}_loss.csv')
# df = pd.read_csv
epochs = df['epoch']
train_loss = df['train_loss']
val_loss = df['val_loss']

# 최솟값 정보
min_train_loss = train_loss.min()
min_train_epoch = epochs[train_loss.idxmin()]
min_val_loss = val_loss.min()
min_val_epoch = epochs[val_loss.idxmin()]

# 그래프 만들기
fig = go.Figure()

# Train Loss
fig.add_trace(go.Scatter(
    x=epochs,
    y=train_loss,
    mode='lines+markers',
    name='Train Loss',
    line=dict(color='blue'),
    hovertemplate='Epoch: %{x}<br>Train Loss: %{y:.4f}<extra></extra>'
))

# Val Loss
fig.add_trace(go.Scatter(
    x=epochs,
    y=val_loss,
    mode='lines+markers',
    name='Val Loss',
    line=dict(color='red'),
    hovertemplate='Epoch: %{x}<br>Val Loss: %{y:.4f}<extra></extra>'
))

# 최소값 표시
fig.add_trace(go.Scatter(
    x=[min_val_epoch],
    y=[min_val_loss],
    mode='markers+text',
    marker=dict(color='green', size=10),
    text=[f"Min Val Loss: {min_val_loss:.4f} (Epoch {min_val_epoch})"],
    textposition='top center',
    name='Min Val',

))

fig.add_trace(go.Scatter(
    x=[min_train_epoch],
    y=[min_train_loss],
    mode='markers+text',
    marker=dict(color='green', size=10),
    text=[f"Min Train Loss: {min_train_loss:.4f} (Epoch {min_train_epoch})"],
    textposition='top center',
    name='Min Train',
))
# 레이아웃 꾸미기
fig.update_layout(
    title=f"Train & Validation Loss_lr={config['learning_rate']}_nlayers={config['n_layers']}",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    hovermode='x unified',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99)
)

fig.show()
